# Full Photometry Pipeline

This tutorial demonstrates how to run the end--to--end photometry pipeline on
real JWST data included with *Mophongo*. We build templates from the F444W
mosaic and fit fluxes in the F770W image using spatially varying PSF
matching kernels.

In [ ]:
from pathlib import Path
import numpy as np
from astropy.io import fits

from mophongo.catalog import Catalog
from mophongo.psf import DrizzlePSF, PSF
from mophongo.psf_map import PSFRegionMap
from mophongo.kernels import KernelLookup
from mophongo.pipeline import run_photometry
from mophongo.templates import Templates

data_dir = Path('data')


## Detect sources in F444W

In [ ]:
sci_444 = data_dir / 'uds-test-f444w_sci.fits'
wht_444 = data_dir / 'uds-test-f444w_wht.fits'
cat = Catalog.from_fits(sci_444, wht_444)
segmap = fits.getdata(data_dir / 'uds-test-LW_seg.fits')
catalog = cat.catalog


## Build PSF region map

In [ ]:
csv_444 = data_dir / 'uds-test-f444w_wcs.csv'
csv_770 = data_dir / 'uds-test-f770w_wcs.csv'
dpsf_444 = DrizzlePSF(driz_image=str(sci_444), csv_file=str(csv_444))
dpsf_770 = DrizzlePSF(driz_image=str(data_dir / 'uds-test-f770w_sci.fits'), csv_file=str(csv_770))

# Keep only footprints overlapping the F444W mosaic
fp = {k: v for k, v in dpsf_770.footprint.items() if v.intersects(dpsf_444.driz_wcs.calc_footprint().astype(float))}
prm = PSFRegionMap.from_footprints(fp)


## Create PSF kernels

In [ ]:
kernels = []
for key, row in prm.regions.iterrows():
    ra, dec = row.geometry.centroid.x, row.geometry.centroid.y
    psf_444 = dpsf_444.get_psf(ra=ra, dec=dec)[1].data
    psf_770 = dpsf_770.get_psf(ra=ra, dec=dec)[1].data
    k = PSF.from_array(psf_444).matching_kernel(psf_770)
    kernels.append(k)
kernels = np.stack(kernels)
klu = KernelLookup(prm, kernels)


## Run photometry

In [ ]:
img_770 = fits.getdata(data_dir / 'uds-test-f770w_sci.fits')
wht_770 = fits.getdata(data_dir / 'uds-test-f770w_wht.fits')
images = [fits.getdata(sci_444), img_770]
psfs = [psf_444, psf_770] = [dpsf_444.get_psf(ra=prm.regions.geometry.centroid.x.iloc[0], dec=prm.regions.geometry.centroid.y.iloc[0])[1].data,
    dpsf_770.get_psf(ra=prm.regions.geometry.centroid.x.iloc[0], dec=prm.regions.geometry.centroid.y.iloc[0])[1].data]
wht = [fits.getdata(wht_444), wht_770]
tbl, resid, _ = run_photometry(images, segmap, catalog, psfs, wht_images=wht, kernels=[None, klu])
tbl.write('photometry.cat', format='ascii.commented_header', overwrite=True)
